Übung : Container-Images und Layers
------------------------------------------------------

Wir wollen das Apache Web Server Image auf docker.io finden.

Dieses Image pullen wir von verschiedenen Registries auf unsere lokale Maschine und analysieren den Inhalt.


In [ ]:
! docker search apache

`docker search` Funktioniert leider nicht mit anderen Registries, darum müssen wir dort die Suche via Web Oberfläche durchführen.

Andere öffentliche Registries sind u.a.
* [AWS Public Registry](https://public.ecr.aws)
* [RedHat](https://quay.io)

Um die Docker "Pull-Raten-Limits" zu umgehen, holen wir den Apache Web Server nicht von `docker.io` sondern von der AWS Public Registry. Dort finden wir, mit den Prefix `docker/library`, praktisch alle "Docker Official Images".

In [ ]:
! docker pull public.ecr.aws/docker/library/httpd
! docker image ls

- - -

### Container Image im `tar` Format speichern

Nicht immer steht eine Image Registry zur Verfügung.

Alternativ kann man Container Images auch von/nach dem [tar](https://wiki.ubuntuusers.de/tar/) Format speichern und laden

In [ ]:
! docker save public.ecr.aws/docker/library/httpd -o httpd.tar
! ls -ls *.tar

Die Datei lässt sich dann, z.B. via USB Stick, auf unsere Zielumgebung transportieren und kann dort wieder importiert werden.

Um diese zu Demonstrieren löschen wir zuerst das gecachte Image weg und importieren dann unser [tar](https://wiki.ubuntuusers.de/tar/) Export.

In [ ]:
%%bash
docker rmi public.ecr.aws/docker/library/httpd
docker image ls
echo "--------------------"
docker load -i httpd.tar
docker image ls

Die erstellte [tar](https://wiki.ubuntuusers.de/tar/)-Datei kann extrahiert werden und so die erstellen Layer analysiert werden.

In [ ]:
! tar xvf httpd.tar
! cat manifest.json | jq

Weitere `tar xvf <uuid>/layer.tar` und `cat <uuid>/json liefern weitere Informationen über das Image.

Probiert es aus, in dem Ihr `[uuid]` durch eine UUID von oben ersetzt:

In [ ]:
! tar xvf [uuid]/layer.tar
! cat [uuid]/json | jq

- - - 

### Container Dateisystem entpacken

Ab und zu, will man nur das Dateisystem des Container erkunden ohne den Container selber zu starten.

Das funktioniert mittels `docker create` statt `run` und Export des Dateisystems:

In [ ]:
! mkdir apache
! docker export $(docker create public.ecr.aws/docker/library/httpd) | tar -C apache -xvf -

Damit wurde ein Verzeichnis `apache` angelegt und dort der Inhalt des Container Images extrahiert.

Mittels `cat` oder `ls -l apache` können wir den Inhalt analysieren

In [ ]:
! cat apache/etc/issue
! echo "-----httpd.conf -------"
! cat apache/usr/local/apache2/conf/httpd.conf

***
Fragen
======

Beantwortet die Fragen einzeln oder in der Gruppe und vergleicht diese mit den Antworten

Was ist der Unterschied zwischen einer Docker Registry und einem Repository
<details><summary>Antwort</summary>
        In der Docker Registry werden die Container Images gespeichert. Ein Repository speichert pro Container Image verschiedene Versionen von Images.
</p></details>

---

Was ist Docker Hub?
<details><summary>Antwort</summary>
        Ein Container Registry, wo Container Image gespeichert werden. Docker Hub wird durch die Firma Docker zur Verfügung gestellt wird.
</p></details>

---

Welches sind die Alternativen?
<details><summary>Antwort</summary>
        Praktisch jeder Cloud Anbieter stellt eine Container Registry zur Verfügung. Auch die Anbieter für die Verwaltung von Build Artefakten (z.B. Sonatype Nexus) stellen Docker Registries zur Verfügung oder haben deren Funktionalität integriert. 
</p></details>

---

Warum sollte eine eigene Container Registry im Unternehmen verwendet werden?
<details><summary>Antwort</summary>
        Sicherheit, bzw. das mögliche Fehlen davon. Es kann nicht Sichergestellt werden, dass alle Container Images auf Docker Hub sicher sind.
</p></details>
